## This code is used to create all the samples used for our Bi Exponential Neural Network! Once all samples are created, DO RESAMPLING NOTEBOOK FIRST, and then use files from resampling for training the Network!

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ffta
import seaborn as sns
from scipy.signal import resample 
from ffta.simulation.mechanical_drive import MechanicalDrive
from ffta.simulation.utils import excitation

c:\users\mdbre\ffta\ffta\pixel_utils\parab.py:8: UserWarning: cuPy not installed; no GPU available
  warnings.warn('cuPy not installed; no GPU available')


In [2]:
can_params = {'amp_invols': 5e-08,
                 'def_invols': 5e-08,
                 'soft_amp': 1.0,
                 'drive_freq': 300000.0,
                 'res_freq': 300000.0,
                 'k':20,
                 'q_factor': 420}
    
force_params = {'es_force': 5e-09,
                 'delta_freq': -30,
                 'tau': [1e-6]}
    
sim_params = {'trigger': 0.0004,
                 'total_time': 0.002,
                 'sampling_rate': 10000000.0}

In [3]:
def add_noise(Z, snr): #Signal-to-noise ratio. Z stands for signal (cantilever deflection)
    sig_avg_power = np.mean(Z**2)
    sig_avg_db = 10*np.log10(sig_avg_power)
    noise_avg_db = sig_avg_db - snr
    noise_avg_power = 10**(noise_avg_db/10)
    noise = np.random.normal(0,np.sqrt(noise_avg_power),len(Z))
    noisy_signal = Z + noise
    return noisy_signal

In [4]:
import datetime

In [5]:
ffts = [60e-6, 70e-6]
drive_freq = []
ks = []
qs = []
tau1s = []
tau2s = []
tfps = []
shifts = []
inst_freqs = []

for i in range(10000):
    k = np.round(np.random.uniform(20,25),1) #K ranges are 20-25
    q = np.round(np.random.uniform(400,450),1) #Q ranges are 400-450
    tau1_base = np.random.uniform(1,50,1) #tau1 ranges are 1-100e-6
    tau1 = tau1_base/1e6
    tau2_base = np.random.uniform(100,500,1) #tau2 ranges are 50-500e-6
    tau2 = tau2_base/1e6
    ks.append(k)
    qs.append(q)
    tau1s.append(tau1_base)
    tau2s.append(tau2_base)
    can_params = {'amp_invols': 5e-08,
                 'def_invols': 5e-08,
                 'soft_amp': 1.0,
                 'drive_freq': 300000,
                 'res_freq': 300000,
                 'k':k,
                 'q_factor': q}
    
    force_params = {'es_force': 5e-09,
                 'delta_freq': -30,
                 'tau': [1e-6]}
    
    sim_params = {'trigger': 0.0004,
                 'total_time': 0.002,
                 'sampling_rate': 10000000.0}
    
    cant = MechanicalDrive(can_params, force_params, sim_params, func=excitation.bi_exp, func_args = [float(tau1), float(tau2)])
    
    Z, info = cant.simulate()
    Z_corr = Z / cant.amp_invols
    
    Z_noisy = add_noise(Z_corr, 60)
    
    fft_ind = np.random.randint(0,2)
    fft_res = ffts[fft_ind]
    
    pix = ffta.pixel.Pixel(Z_noisy, method='stft',trigger=0.0004,total_time=0.002, roi=1e-3)
    pix.fft_time_res = fft_res
    _ = pix.analyze()
    
    tfps.append(pix.tfp)
    shifts.append(pix.shift)
    inst_freqs.append(pix.inst_freq[int(0.5*0.0004*1e7):int(0.9*20000)])
    

    if ((i+1)%100) == 0:
        date = datetime.date.today().strftime("%Y%m%d")
        np.save(date+'_ks.npy', ks)
        np.save(date+'_qs.npy', qs)
        np.save(date+'_tau1s.npy',tau1s)
        np.save(date+'_tau2s.npy',tau2s)
        np.save(date+'_tfps.npy', tfps)
        np.save(date+'_shifts.npy', shifts)
        np.save(date+'_instfreqs.npy',inst_freqs)
        np.save(date+'_drive_freqs.npy', drive_freq)